# Propose centroid replacing the masters with BERT siamese

In [1]:
# import tensorflow as tf
import keras
# from tensorflow.python import keras
import os

Using TensorFlow backend.


In [2]:
# Optimize the use of GPUs
# https://datascience.stackexchange.com/questions/23895/multi-gpu-in-keras
# https://keras.io/getting-started/faq/#how-can-i-run-a-keras-model-on-multiple-gpus
# https://stackoverflow.com/questions/56316451/how-to-use-specific-gpus-in-keras-for-multi-gpu-training

In [3]:
from __future__ import print_function, division

In [4]:
import re
import numpy as np
import pandas as pd

import os
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sys
from annoy import AnnoyIndex
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%matplotlib inline

In [5]:
from keras.layers import Conv1D, Input, Add, Activation, Dropout, Embedding, MaxPooling1D, \
    GlobalMaxPool1D, Flatten, Dense, Concatenate, BatchNormalization
from keras.models import Sequential, Model
from keras.regularizers import l2
from keras.initializers import TruncatedNormal
from keras import optimizers

In [6]:
from methods.baseline import Baseline
from methods.experiments import Experiment
from methods.evaluation import Evaluation
from methods.retrieval import Retrieval

## Auxiliary methods

## Configurações Globais

In [7]:
MAX_SEQUENCE_LENGTH_T = 20 # 20
MAX_SEQUENCE_LENGTH_D = 20 # 80
EMBEDDING_DIM = 300
MAX_NB_WORDS = 20000

'''
    Configuration
'''
epochs = 1000
best_loss = 1
best_epoch = 0
verbose = 0
loss = 1

### Parse bugs preproprecessed

In [8]:
# Domain to use
DOMAIN = 'eclipse'
METHOD = 'propose_centroid_bert_{}'.format(epochs)
# Dataset paths
DIR = 'data/processed/{}'.format(DOMAIN)
DIR_PAIRS = 'data/normalized/{}'.format(DOMAIN)
DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
# Path embeddings
EMBED_DIR='data/embed'
# Save model
SAVE_PATH = '{}_feature@number_of_epochs@epochs_64batch({})'.format(METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_feature_@number_of_epochs@epochs_64batch({})'.format(METHOD, DOMAIN)

# Extract CORPUs
EXTRACT_CORPUS = False

In [9]:
# !wget -q https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
# !unzip -o uncased_L-12_H-768_A-12.zip

In [10]:
import os

pretrained_path = 'uncased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
model_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [11]:
from keras_bert import load_vocabulary

token_dict = load_vocabulary(vocab_path)

In [12]:
"Total vocabulary: {}".format(len(token_dict))

'Total vocabulary: 30522'

In [13]:
baseline = Baseline(DOMAIN, DIR, DATASET, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 
                    token_dict['[CLS]'], token_dict['[SEP]'])
evaluation = Evaluation(verbose=0)
retrieval = Retrieval()
experiment = Experiment(baseline, evaluation)

In [14]:
experiment.set_retrieval(retrieval, baseline, DOMAIN)

Creating the buckets...


#### Loading bug ids in memory

In [15]:
experiment.load_ids()
len(baseline.bug_ids)

Reading bug ids


361006

#### Dicionário de títulos e descrições

In [16]:
%%time

experiment.load_bugs()
len(baseline.sentence_dict)


CPU times: user 30.9 s, sys: 3.03 s, total: 33.9 s
Wall time: 33.8 s


#### Hashing bugs by buckets

In [17]:
issues_by_buckets = experiment.get_buckets_for_bugs()

#### Prepare the train and test

In [18]:
%%time

experiment.prepare_dataset(issues_by_buckets, path_train='train_chronological', path_test='test_chronological')
# Read and create the test queries duplicates
retrieval.create_queries()

Reading train data
Reading bug ids
CPU times: user 2min 36s, sys: 20.5 ms, total: 2min 36s
Wall time: 2min 36s


#### Recovery bug ids from train

In [19]:
bug_train_ids = experiment.get_train_ids(baseline.train_data)

#### Display a random bug

In [20]:
idx = np.random.choice(baseline.bug_ids, 1)[0]
baseline.bug_set[idx]

{'bug_severity': '2\n',
 'bug_status': '2\n',
 'component': '343\n',
 'creation_ts': '2004-08-11 16:55:00 -0400',
 'delta_ts': '2012-02-15 13:47:10 -0500',
 'description': "[CLS] when i create the ur ##l test sample , i get build errors in the project . the reason is that it ' s referencing some jars in other plug ##ins with specific versions ( 3 . 0 . 0 ) , but the plug ##in versions have been updated to 3 . 0 . 1 so the jar references are invalid . perhaps a better system is needed , other than direct references , because this problem will keep popping up in the future . [SEP]",
 'description_bert': "[CLS] when i create the ur ##l test sample , i get build errors in the project . the reason is that it ' s referencing some jars in other plug ##ins with specific versions ( 3 . 0 . 0 ) , but the plug ##in versions have been updated to 3 . 0 . 1 so the jar references are invalid . perhaps a better system is needed , other than direct references , because this problem will keep popping up

### Generating the batch test

In [21]:
"Train ", len(baseline.dup_sets_train)

('Train ', 34882)

In [22]:
bug_idx = bug_train_ids[0]
vector = baseline.bug_set[bug_idx]['textual_word']
annoy_train = AnnoyIndex(vector.shape[0])
for bug_id in bug_train_ids:
    annoy_train.add_item(bug_id, baseline.bug_set[bug_id]['textual_word'])
annoy_train.build(10) # 10 trees
"Indexed all train"

'Indexed all train'

### Train ids

In [23]:
bug_train_ids = experiment.get_train_ids(baseline.train_data)

In [24]:
%%time

batch_size = 64
batch_size_test = 128

# we want a constant validation group to have a frame of reference for model performance
batch_triplets_valid, valid_input_sample, valid_input_pos, valid_input_neg, \
                            valid_master_sample, valid_master_neg, valid_sim = experiment.batch_iterator_bert(None, baseline.train_data, 
                                                                                          baseline.dup_sets_train,
                                                                                          bug_train_ids,
                                                                                          batch_size_test, 1, 
                                                                                              issues_by_buckets, INCLUDE_MASTER=True)

# Categorical columns
number_of_columns_info = valid_input_sample['info'].shape[1]
# Max sequence title
MAX_SEQUENCE_LENGTH_T = valid_input_sample['title']['token'].shape[1]
MAX_SEQUENCE_LENGTH_D = valid_input_sample['description']['token'].shape[1]

CPU times: user 66.4 ms, sys: 0 ns, total: 66.4 ms
Wall time: 66.1 ms


In [25]:
valid_input_sample['title']['token'].shape, valid_input_sample['description']['token'].shape, \
    valid_input_sample['info'].shape, valid_sim.shape

((128, 20), (128, 20), (128, 1682), (128,))

### Validar entrada

In [26]:
# %%time 

# baseline.display_batch(baseline.train_data, baseline.dup_sets_train, bug_train_ids, 5, batch_iterator, issues_by_buckets)

## Experiment

## Propose

https://github.com/tqtg/DuplicateBugFinder

In [27]:
from keras.initializers import RandomUniform, RandomNormal, Ones

### BERT

https://github.com/CyberZHG/keras-bert

In [28]:
from keras_bert import load_trained_model_from_checkpoint
from keras_bert import compile_model, get_model
from keras.layers import GlobalAveragePooling1D

def bert_model(MAX_SEQUENCE_LENGTH, name):
    layer_num = 8
#     model = load_trained_model_from_checkpoint(
#             config_path,
#             model_path,
#             training=True,
#             trainable=True,
#             seq_len=MAX_SEQUENCE_LENGTH,
#     )
    model = load_trained_model_from_checkpoint(
        config_path,
        model_path,
        training=True,
        use_adapter=True,
        seq_len=MAX_SEQUENCE_LENGTH,
        trainable=['Encoder-{}-MultiHeadSelfAttention-Adapter'.format(i + 1) for i in range(12-layer_num, 13)] +
        ['Encoder-{}-FeedForward-Adapter'.format(i + 1) for i in range(12-layer_num, 13)] +
        ['Encoder-{}-MultiHeadSelfAttention-Norm'.format(i + 1) for i in range(12-layer_num, 13)] +
        ['Encoder-{}-FeedForward-Norm'.format(i + 1) for i in range(layer_num)],
    )
#     model = get_model(
#         token_num=len(token_dict),
#         head_num=10,
#         transformer_num=layer_num,
#         embed_dim=100,
#         feed_forward_dim=100,
#         seq_len=MAX_SEQUENCE_LENGTH,
#         pos_num=MAX_SEQUENCE_LENGTH,
#         dropout_rate=0.05,
#     )
    compile_model(model)
    inputs = model.inputs[:2]
    outputs = model.get_layer('Encoder-{}-FeedForward-Norm'.format(layer_num)).output
    #outputs = model.get_layer('Extract').output
    outputs = GlobalAveragePooling1D()(outputs)
#     outputs = Dense(300, activation='tanh')(outputs)
    
    model = Model(inputs, outputs, name='FeatureBERTGenerationModel{}'.format(name))
    
    return model

### MLP

In [29]:
def mlp_model(input_size):
    info_input = Input(shape=(input_size, ), name='Feature_BugInput')
    input_size = 300
    
    layer = Dense(input_size, activation='tanh')(info_input)
    
    #layer = GRU(100, activation='tanh')(layer)
    
    mlp_feature_model = Model(inputs=[info_input], outputs=[layer], name = 'FeatureMlpGenerationModel')
    
    return mlp_feature_model

### Siamese model

In [30]:
from keras import backend as K

'''
    Some loss ideas
    hinge loss Kullback-Leibler
    https://stackoverflow.com/questions/53581298/custom-combined-hinge-kb-divergence-loss-function-in-siamese-net-fails-to-genera
'''

def normalize(x, axis):
    norm = K.sqrt(K.sum(K.square(x), axis=axis, keepdims=False))
    return x, K.maximum(norm, K.epsilon())
    
# https://github.com/keras-team/keras/issues/3031
# https://github.com/keras-team/keras/issues/8335
def cosine_distance(inputs):
    x, y = inputs
    x, x_norm = normalize(x, axis=-1)
    y, y_norm = normalize(y, axis=-1)
    distance = K.sum( x * y, axis=-1) / (x_norm * y_norm)
    distance = (distance + K.constant(1)) / K.constant(2)
    # Distance goes from 0 to 2 in theory, but from 0 to 1 if x and y are both
    # positive (which is the case after ReLU activation).
    #return K.mean(distance, axis=-1, keepdims=False)
    return K.mean(distance, axis=-1, keepdims=True)
    
def euclidean_distance(vects):
    x, y = vects
    distance = K.sqrt(K.maximum(K.sum(K.square(x - y), axis=1, keepdims=True), K.epsilon()))
    # Normalize https://stats.stackexchange.com/questions/53068/euclidean-distance-score-and-similarity
    distance = K.constant(1) / (K.constant(1) + distance)
    return K.mean(distance, keepdims=False)
    #return K.mean(distance, axis=-1, keepdims=False)

# https://jdhao.github.io/2017/03/13/some_loss_and_explanations/
def _triplet_loss(vects):
    margin = K.constant(1.0)
    pos = vects[0]
    neg = vects[1]
    return K.mean(K.maximum(0.0, pos - neg + margin))

def triplet_loss(vects):
    margin = K.constant(1.0)
    pos = vects[0]
    neg = vects[1]
    return K.maximum(0.0, margin - pos + neg)

# https://www.kaggle.com/c/quora-question-pairs/discussion/33631
# https://www.researchgate.net/figure/Illustration-of-triplet-loss-contrastive-loss-for-negative-samples-and-binomial_fig2_322060548
def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    pos = y_pred[0]
    neg = y_pred[1]
    margin = 1
    return K.mean(pos * K.square(neg) +
                  (1 - pos) * K.square(K.maximum(margin - neg, 0)))

def custom_loss(y_true, y_pred):
    return y_pred[0]
def anchor_pos_distance(y_true, y_pred):
    return y_pred[1]
def anchor_neg_distance(y_true, y_pred):
    return y_pred[2]
def anchor_centroid_pos_distance(y_true, y_pred):
    return y_pred[3]
def anchor_centroid_neg_distance(y_true, y_pred):
    return y_pred[4]
def pos_centroid_pos_distance(y_true, y_pred):
    return y_pred[5]
def pos_centroid_neg_distance(y_true, y_pred):
    return y_pred[6]
def neg_centroid_pos_distance(y_true, y_pred): 
    return y_pred[7]
def neg_centroid_neg_distance(y_true, y_pred):
    return y_pred[8]
def triplet_bug(y_true, y_pred):
    return y_pred[9]
def triplet_anchor(y_true, y_pred):
    return y_pred[10]
def triplet_pos(y_true, y_pred):
    return y_pred[11]
def triplet_neg(y_true, y_pred):
    return y_pred[12]
def stack_tensors(vects):
    return K.stack(vects, axis=-1)

### Propose

In [31]:
from keras.layers import concatenate, Add, Lambda, Average, Maximum, Subtract, Average, AveragePooling1D, GlobalAveragePooling1D
from keras.optimizers import Adam, Nadam

def siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, sequence_length_info, 
                  sequence_length_t, sequence_length_d, name):
    
    # Title
    bug_t_token = Input(shape = (sequence_length_t, ), name = 'title_token_{}'.format(name))
    bug_t_segment = Input(shape = (sequence_length_t, ), name = 'title_segment_{}'.format(name))
    # Description
    bug_d_token = Input(shape = (sequence_length_d, ), name = 'desc_token_{}'.format(name))
    bug_d_segment = Input(shape = (sequence_length_d, ), name = 'desc_segment_{}'.format(name))
    # Categorical
    bug_i = Input(shape = (sequence_length_info, ), name = 'info_{}'.format(name))
    
    bug_t_feat = title_feature_model([bug_t_token, bug_t_segment])
    bug_d_feat = desc_feature_model([bug_d_token, bug_d_segment])
    bug_i_feat = categorical_feature_model(bug_i)
    
    #bug_feature_output = Add(name = 'merge_features_{}'.format(name))([bug_i_feat, bug_t_feat, bug_d_feat])
    bug_feature_output = concatenate([bug_i_feat, bug_t_feat, bug_d_feat], name = 'merge_features_{}'.format(name))
    
    bug_feature_model = Model(inputs=[bug_t_token, bug_t_segment, bug_d_token, bug_d_segment, bug_i], outputs=[bug_feature_output], name = 'merge_features_{}'.format(name))
    
    return bug_feature_model

In [32]:
def siamese_model_centroid(sequence_length, name):
    
    # Bug Centroid Feature
    bug_centroid = Input(shape = (sequence_length, ), name = 'bug_centroid_feature_{}'.format(name))
    
    bug_feature_model = Model(inputs=[bug_centroid], outputs=[bug_centroid], name = 'merge_features_{}'.format(name))
    
    return bug_feature_model

In [33]:
from keras.layers import Average, Reshape, Add
from keras_radam import RAdam
from keras_bert import AdamWarmup, calc_train_steps

def max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, 
                             master_anchor, master_negative, master_positive, 
                         NUMBER_OF_INSTANCES, BATCH_SIZE, EPOCHS, decay_lr=1):
    
    inputs = np.concatenate([encoded_anchor.input, encoded_positive.input, encoded_negative.input], -1).tolist()
    
    inputs.append(master_anchor.input)
    inputs.append(master_positive.input)
    inputs.append(master_negative.input)
    
    encoded_anchor = encoded_anchor.output
    encoded_positive = encoded_positive.output
    encoded_negative = encoded_negative.output
    master_anchor = master_anchor.output
    master_negative = master_negative.output
    master_positive = master_positive.output
    
    # Distance bugs
    positive_d = Lambda(cosine_distance, name='pos_cosine_distance')([encoded_anchor, encoded_positive])
    negative_d = Lambda(cosine_distance, name='neg_cosine_distance')([encoded_anchor, encoded_negative])
    
    # Distance masters anchor
    master_anchor_positive_d = Lambda(cosine_distance, name='pos_master_cosine_distance')([encoded_anchor, master_positive])
    master_anchor_negative_d = Lambda(cosine_distance, name='neg_master_cosine_distance')([encoded_anchor, master_negative])
    
    # Distance master positive
    master_pos_positive_d = Lambda(cosine_distance, name='pos_master_pos_cosine_distance')([encoded_positive, master_positive])
    master_pos_negative_d = Lambda(cosine_distance, name='neg_master_pos_cosine_distance')([encoded_positive, master_negative])
    
    # Distance master negative
    master_neg_positive_d = Lambda(cosine_distance, name='pos_master_neg_cosine_distance')([encoded_negative, master_negative])
    master_neg_negative_d = Lambda(cosine_distance, name='neg_master_neg_cosine_distance')([encoded_negative, master_positive])
    

    # Loss function only works with a single output
    output_bug = Lambda(triplet_loss, name='triplet_pos_neg')([positive_d, negative_d])
    output_master = Lambda(triplet_loss, name='triplet_anchor_centroid')([master_anchor_positive_d, master_anchor_negative_d])
    output_master_pos = Lambda(triplet_loss, name='triplet_pos_centroid')([master_pos_positive_d, master_pos_negative_d])
    output_master_neg = Lambda(triplet_loss, name='triplet_neg_centroid')([master_neg_positive_d, master_neg_negative_d])
    
#     output_bug = Reshape((1, ))(output_bug)
#     output_anchor = Reshape((1, ))(output_master)
#     output_pos = Reshape((1, ))(output_master_pos)
#     output_neg = Reshape((1, ))(output_master_neg)

    output_anchor = output_master
    output_pos = output_master_pos
    output_neg = output_master_neg
    
    output_median = Average()([output_bug, output_anchor, output_pos, output_neg])
    
    output = concatenate([output_median, positive_d, negative_d, 
                          master_anchor_positive_d, master_anchor_negative_d,
                         master_pos_positive_d, master_pos_negative_d,
                         master_neg_positive_d, master_neg_negative_d,
                         output_bug, output_master, output_master_pos, output_master_neg])

    similarity_model = Model(inputs = inputs, outputs = [output], name = 'Similarity_Model')

    # setup the optimization process 
    similarity_model.compile(optimizer='adam', loss=custom_loss, 
                                 metrics=[anchor_pos_distance, anchor_neg_distance, 
                                          anchor_centroid_pos_distance, anchor_centroid_neg_distance,
                                         pos_centroid_pos_distance, pos_centroid_neg_distance,
                                         neg_centroid_pos_distance, neg_centroid_neg_distance,
                                         triplet_bug, triplet_anchor, triplet_pos, triplet_neg])

    return similarity_model

In [ ]:
%%time
import os

print("Batch size ", batch_size)

# Inspired on https://'pastebin.com/TaGFdcBA
keras.backend.clear_session()

# Feature models
'''
    cnn_dilated_model
    arcii_model
    cnn_model
    lstm_model
    bilstm_model
'''
# title_feature_model = bilstm_model(title_embedding_layer, MAX_SEQUENCE_LENGTH_T)
title_feature_model = bert_model(MAX_SEQUENCE_LENGTH_T, 'Title')
desc_feature_model = bert_model(MAX_SEQUENCE_LENGTH_D, 'Description')
#desc_feature_model = cnn_model(desc_embedding_layer, MAX_SEQUENCE_LENGTH_D)
categorical_feature_model = mlp_model(number_of_columns_info)

# Similarity model
encoded_anchor = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'in')
encoded_positive = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'pos')
encoded_negative = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'neg')
# Master model
embed_size = K.int_shape(title_feature_model.get_output_at(0))[1] + K.int_shape(desc_feature_model.get_output_at(0))[1] + K.int_shape(categorical_feature_model.get_output_at(0))[1] 

master_anchor = siamese_model_centroid(embed_size, 'master_anchor')
master_pos = siamese_model_centroid(embed_size, 'master_pos')
master_negative = siamese_model_centroid(embed_size, 'master_neg')

NUMBER_OF_INSTANCES = len(baseline.dup_sets_train)
BATCH_SIZE = batch_size
EPOCHS = epochs

similarity_model = max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, 
                                            master_anchor, master_negative, master_pos,
                                            NUMBER_OF_INSTANCES, BATCH_SIZE, EPOCHS, decay_lr=1)

# cnn_feature_model.summary()
# lstm_feature_model.summary()
similarity_model.summary()

'''
    Experiment
'''
for epoch in range(epochs):
    batch_triplet_train, \
        train_input_sample, train_input_pos, train_input_neg, train_master_input, train_master_neg, \
            train_sim = experiment.batch_iterator_bert(encoded_anchor, baseline.train_data, baseline.dup_sets_train, \
                                                       bug_train_ids, 
                                                       batch_size, 1, 
                                                       issues_by_buckets, 
                                                       TRIPLET_HARD=True, USE_CENTROID=True)
    
    train_batch = [train_input_sample['title']['token'], train_input_sample['title']['segment'], train_input_sample['description']['token'], train_input_sample['description']['segment'], train_input_sample['info'],
                   train_input_pos['title']['token'], train_input_pos['title']['segment'], train_input_pos['description']['token'], train_input_pos['description']['segment'], train_input_pos['info'], 
                   train_input_neg['title']['token'], train_input_neg['title']['segment'], train_input_neg['description']['token'], train_input_neg['description']['segment'], train_input_neg['info'],
                   train_master_input['centroid_embed'],
                   train_master_input['centroid_embed'],
                   train_master_neg['centroid_embed']]
    
#     if epoch == 10:
#         similarity_model = max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, decay_lr=0.1)
    
    h = similarity_model.train_on_batch(x=train_batch, y=train_sim)
    
    if (epoch+1 == epochs): #(epoch > 1 and epoch % 10 == 0) or (epoch+1 == epochs):
        recall, _, debug = experiment.evaluate_validation_test(retrieval, verbose, encoded_anchor, issues_by_buckets, 
                                                               bug_train_ids, method='bert')
        print(("Epoch: {}\n" + 
              "Loss: {:.2f}, pos_cosine: {:.2f}, neg_cosine: {:.2f}, TL: {:.2f}\n" +
              "anchor_pos_centroid_cosine: {:.2f}, anchor_neg_centroid_cosine: {:.2f}, TL_A: {:.2f}\n" +
              "pos_pos_centroid_cosine: {:.2f}, pos_neg_centroid_cosine: {:.2f}, TL_A: {:.2f}\n" +
              "neg_pos_centroid_cosine: {:.2f}, neg_neg_centroid_cosine: {:.2f}, TL_N: {:.2f}, recall@25: {:.2f}").format(epoch+1,
                                                                                                         h[0],
                                                                                                         h[1], h[2], h[9], 
                                                                                                         h[3], h[4], h[10],
                                                                                                         h[5], h[6], h[11],
                                                                                                         h[7], h[8], h[12],
                                                                                                         recall))
    else:
        print(("Epoch: {}\n" + 
              "Loss: {:.2f}, pos_cosine: {:.2f}, neg_cosine: {:.2f}, TL: {:.2f}\n" +
              "anchor_pos_centroid_cosine: {:.2f}, anchor_neg_centroid_cosine: {:.2f}, TL_A: {:.2f}\n" +
              "pos_pos_centroid_cosine: {:.2f}, pos_neg_centroid_cosine: {:.2f}, TL_P: {:.2f}\n" +
              "neg_pos_centroid_cosine: {:.2f}, neg_neg_centroid_cosine: {:.2f}, TL_N: {:.2f}").format(epoch+1,
                                                                                                         h[0],
                                                                                                         h[1], h[2], h[9], 
                                                                                                         h[3], h[4], h[10],
                                                                                                         h[5], h[6], h[11],
                                                                                                         h[7], h[8], h[12]))
    loss = h[0]
    
    if loss < best_loss:
        best_loss = loss
        best_epoch = epoch+1

experiment.save_model(similarity_model, SAVE_PATH.replace('@number_of_epochs@', str(epochs)))
experiment.save_model(encoded_anchor, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)), verbose=1)
print('Best_epoch={}, Best_loss={:.2f}s, Recall@25={:.2f}'.format(best_epoch, best_loss, recall))

Batch size  64
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
info_in (InputLayer)            (None, 1682)         0                                            
__________________________________________________________________________________________________
title_token_in (InputLayer)     (None, 20)           0                                            
__________________________________________________________________________________________________
title_segment_in (InputLayer)   (None, 20)           0                                            
__________________________________________________________________________________________________
desc_token_in (InputLayer)      (None, 20)           0                                            
______________________________________________________________________________________________

Epoch: 1
Loss: 0.97, pos_cosine: 0.90, neg_cosine: 0.91, TL: 1.01
anchor_pos_centroid_cosine: 0.92, anchor_neg_centroid_cosine: 0.89, TL_A: 0.96
pos_pos_centroid_cosine: 0.92, pos_neg_centroid_cosine: 0.88, TL_P: 0.96
neg_pos_centroid_cosine: 0.93, neg_neg_centroid_cosine: 0.90, TL_N: 0.97
Epoch: 2
Loss: 0.97, pos_cosine: 0.89, neg_cosine: 0.90, TL: 1.01
anchor_pos_centroid_cosine: 0.92, anchor_neg_centroid_cosine: 0.88, TL_A: 0.96
pos_pos_centroid_cosine: 0.92, pos_neg_centroid_cosine: 0.88, TL_P: 0.96
neg_pos_centroid_cosine: 0.93, neg_neg_centroid_cosine: 0.89, TL_N: 0.96
Epoch: 3
Loss: 0.98, pos_cosine: 0.88, neg_cosine: 0.89, TL: 1.02
anchor_pos_centroid_cosine: 0.91, anchor_neg_centroid_cosine: 0.88, TL_A: 0.97
pos_pos_centroid_cosine: 0.91, pos_neg_centroid_cosine: 0.87, TL_P: 0.95
neg_pos_centroid_cosine: 0.92, neg_neg_centroid_cosine: 0.89, TL_N: 0.97
Epoch: 4
Loss: 0.97, pos_cosine: 0.88, neg_cosine: 0.89, TL: 1.01
anchor_pos_centroid_cosine: 0.91, anchor_neg_centroid_cosine:

Epoch: 32
Loss: 0.89, pos_cosine: 0.69, neg_cosine: 0.68, TL: 0.99
anchor_pos_centroid_cosine: 0.81, anchor_neg_centroid_cosine: 0.67, TL_A: 0.86
pos_pos_centroid_cosine: 0.81, pos_neg_centroid_cosine: 0.65, TL_P: 0.84
neg_pos_centroid_cosine: 0.81, neg_neg_centroid_cosine: 0.69, TL_N: 0.87
Epoch: 33
Loss: 0.91, pos_cosine: 0.66, neg_cosine: 0.67, TL: 1.01
anchor_pos_centroid_cosine: 0.79, anchor_neg_centroid_cosine: 0.67, TL_A: 0.88
pos_pos_centroid_cosine: 0.80, pos_neg_centroid_cosine: 0.65, TL_P: 0.85
neg_pos_centroid_cosine: 0.80, neg_neg_centroid_cosine: 0.68, TL_N: 0.88
Epoch: 34
Loss: 0.89, pos_cosine: 0.69, neg_cosine: 0.67, TL: 0.98
anchor_pos_centroid_cosine: 0.80, anchor_neg_centroid_cosine: 0.66, TL_A: 0.86
pos_pos_centroid_cosine: 0.80, pos_neg_centroid_cosine: 0.64, TL_P: 0.84
neg_pos_centroid_cosine: 0.80, neg_neg_centroid_cosine: 0.68, TL_N: 0.88
Epoch: 35
Loss: 0.89, pos_cosine: 0.68, neg_cosine: 0.67, TL: 0.99
anchor_pos_centroid_cosine: 0.80, anchor_neg_centroid_cos

Epoch: 61
Loss: 0.87, pos_cosine: 0.63, neg_cosine: 0.62, TL: 0.99
anchor_pos_centroid_cosine: 0.77, anchor_neg_centroid_cosine: 0.61, TL_A: 0.83
pos_pos_centroid_cosine: 0.78, pos_neg_centroid_cosine: 0.58, TL_P: 0.80
neg_pos_centroid_cosine: 0.76, neg_neg_centroid_cosine: 0.62, TL_N: 0.86
Epoch: 62
Loss: 0.85, pos_cosine: 0.64, neg_cosine: 0.63, TL: 0.98
anchor_pos_centroid_cosine: 0.79, anchor_neg_centroid_cosine: 0.62, TL_A: 0.82
pos_pos_centroid_cosine: 0.79, pos_neg_centroid_cosine: 0.57, TL_P: 0.78
neg_pos_centroid_cosine: 0.79, neg_neg_centroid_cosine: 0.61, TL_N: 0.82
Epoch: 63
Loss: 0.86, pos_cosine: 0.63, neg_cosine: 0.63, TL: 1.00
anchor_pos_centroid_cosine: 0.79, anchor_neg_centroid_cosine: 0.62, TL_A: 0.83
pos_pos_centroid_cosine: 0.79, pos_neg_centroid_cosine: 0.57, TL_P: 0.78
neg_pos_centroid_cosine: 0.79, neg_neg_centroid_cosine: 0.62, TL_N: 0.83
Epoch: 64
Loss: 0.85, pos_cosine: 0.63, neg_cosine: 0.61, TL: 0.99
anchor_pos_centroid_cosine: 0.78, anchor_neg_centroid_cos

Epoch: 90
Loss: 0.85, pos_cosine: 0.63, neg_cosine: 0.62, TL: 0.99
anchor_pos_centroid_cosine: 0.78, anchor_neg_centroid_cosine: 0.60, TL_A: 0.82
pos_pos_centroid_cosine: 0.79, pos_neg_centroid_cosine: 0.56, TL_P: 0.77
neg_pos_centroid_cosine: 0.77, neg_neg_centroid_cosine: 0.60, TL_N: 0.84
Epoch: 91
Loss: 0.85, pos_cosine: 0.63, neg_cosine: 0.61, TL: 0.98
anchor_pos_centroid_cosine: 0.79, anchor_neg_centroid_cosine: 0.60, TL_A: 0.81
pos_pos_centroid_cosine: 0.79, pos_neg_centroid_cosine: 0.57, TL_P: 0.78
neg_pos_centroid_cosine: 0.77, neg_neg_centroid_cosine: 0.60, TL_N: 0.82
Epoch: 92
Loss: 0.86, pos_cosine: 0.64, neg_cosine: 0.63, TL: 0.99
anchor_pos_centroid_cosine: 0.78, anchor_neg_centroid_cosine: 0.61, TL_A: 0.83
pos_pos_centroid_cosine: 0.79, pos_neg_centroid_cosine: 0.57, TL_P: 0.78
neg_pos_centroid_cosine: 0.79, neg_neg_centroid_cosine: 0.62, TL_N: 0.82
Epoch: 93
Loss: 0.84, pos_cosine: 0.66, neg_cosine: 0.62, TL: 0.96
anchor_pos_centroid_cosine: 0.79, anchor_neg_centroid_cos

...

Epoch: 538
Loss: 0.84, pos_cosine: 0.68, neg_cosine: 0.65, TL: 0.97
anchor_pos_centroid_cosine: 0.84, anchor_neg_centroid_cosine: 0.64, TL_A: 0.80
pos_pos_centroid_cosine: 0.83, pos_neg_centroid_cosine: 0.61, TL_P: 0.78
neg_pos_centroid_cosine: 0.84, neg_neg_centroid_cosine: 0.66, TL_N: 0.82
Epoch: 539
Loss: 0.82, pos_cosine: 0.70, neg_cosine: 0.64, TL: 0.94
anchor_pos_centroid_cosine: 0.85, anchor_neg_centroid_cosine: 0.63, TL_A: 0.78
pos_pos_centroid_cosine: 0.85, pos_neg_centroid_cosine: 0.60, TL_P: 0.76
neg_pos_centroid_cosine: 0.84, neg_neg_centroid_cosine: 0.64, TL_N: 0.80
Epoch: 540
Loss: 0.80, pos_cosine: 0.72, neg_cosine: 0.64, TL: 0.92
anchor_pos_centroid_cosine: 0.86, anchor_neg_centroid_cosine: 0.63, TL_A: 0.76
pos_pos_centroid_cosine: 0.87, pos_neg_centroid_cosine: 0.60, TL_P: 0.73
neg_pos_centroid_cosine: 0.85, neg_neg_centroid_cosine: 0.64, TL_N: 0.79
Epoch: 541
Loss: 0.82, pos_cosine: 0.70, neg_cosine: 0.64, TL: 0.93
anchor_pos_centroid_cosine: 0.85, anchor_neg_centroid

Epoch: 566
Loss: 0.81, pos_cosine: 0.70, neg_cosine: 0.64, TL: 0.94
anchor_pos_centroid_cosine: 0.85, anchor_neg_centroid_cosine: 0.63, TL_A: 0.79
pos_pos_centroid_cosine: 0.86, pos_neg_centroid_cosine: 0.59, TL_P: 0.73
neg_pos_centroid_cosine: 0.85, neg_neg_centroid_cosine: 0.65, TL_N: 0.79
Epoch: 567
Loss: 0.80, pos_cosine: 0.71, neg_cosine: 0.63, TL: 0.92
anchor_pos_centroid_cosine: 0.86, anchor_neg_centroid_cosine: 0.63, TL_A: 0.77
pos_pos_centroid_cosine: 0.86, pos_neg_centroid_cosine: 0.59, TL_P: 0.73
neg_pos_centroid_cosine: 0.86, neg_neg_centroid_cosine: 0.63, TL_N: 0.77
Epoch: 568
Loss: 0.83, pos_cosine: 0.71, neg_cosine: 0.65, TL: 0.94
anchor_pos_centroid_cosine: 0.84, anchor_neg_centroid_cosine: 0.65, TL_A: 0.80
pos_pos_centroid_cosine: 0.85, pos_neg_centroid_cosine: 0.61, TL_P: 0.75
neg_pos_centroid_cosine: 0.84, neg_neg_centroid_cosine: 0.65, TL_N: 0.81
Epoch: 569
Loss: 0.82, pos_cosine: 0.72, neg_cosine: 0.65, TL: 0.93
anchor_pos_centroid_cosine: 0.85, anchor_neg_centroid

Epoch: 594
Loss: 0.81, pos_cosine: 0.70, neg_cosine: 0.64, TL: 0.94
anchor_pos_centroid_cosine: 0.86, anchor_neg_centroid_cosine: 0.64, TL_A: 0.77
pos_pos_centroid_cosine: 0.85, pos_neg_centroid_cosine: 0.61, TL_P: 0.76
neg_pos_centroid_cosine: 0.86, neg_neg_centroid_cosine: 0.65, TL_N: 0.79
Epoch: 595
Loss: 0.80, pos_cosine: 0.73, neg_cosine: 0.64, TL: 0.91
anchor_pos_centroid_cosine: 0.86, anchor_neg_centroid_cosine: 0.64, TL_A: 0.77
pos_pos_centroid_cosine: 0.87, pos_neg_centroid_cosine: 0.61, TL_P: 0.74
neg_pos_centroid_cosine: 0.87, neg_neg_centroid_cosine: 0.65, TL_N: 0.78
Epoch: 596
Loss: 0.81, pos_cosine: 0.73, neg_cosine: 0.65, TL: 0.93
anchor_pos_centroid_cosine: 0.87, anchor_neg_centroid_cosine: 0.65, TL_A: 0.78
pos_pos_centroid_cosine: 0.86, pos_neg_centroid_cosine: 0.59, TL_P: 0.73
neg_pos_centroid_cosine: 0.86, neg_neg_centroid_cosine: 0.65, TL_N: 0.79
Epoch: 597
Loss: 0.83, pos_cosine: 0.71, neg_cosine: 0.66, TL: 0.94
anchor_pos_centroid_cosine: 0.85, anchor_neg_centroid

Epoch: 622
Loss: 0.82, pos_cosine: 0.69, neg_cosine: 0.64, TL: 0.95
anchor_pos_centroid_cosine: 0.84, anchor_neg_centroid_cosine: 0.62, TL_A: 0.78
pos_pos_centroid_cosine: 0.83, pos_neg_centroid_cosine: 0.58, TL_P: 0.75
neg_pos_centroid_cosine: 0.82, neg_neg_centroid_cosine: 0.63, TL_N: 0.81
Epoch: 623
Loss: 0.84, pos_cosine: 0.71, neg_cosine: 0.66, TL: 0.95
anchor_pos_centroid_cosine: 0.85, anchor_neg_centroid_cosine: 0.66, TL_A: 0.80
pos_pos_centroid_cosine: 0.84, pos_neg_centroid_cosine: 0.61, TL_P: 0.78
neg_pos_centroid_cosine: 0.83, neg_neg_centroid_cosine: 0.67, TL_N: 0.83
Epoch: 624
Loss: 0.82, pos_cosine: 0.72, neg_cosine: 0.66, TL: 0.94
anchor_pos_centroid_cosine: 0.86, anchor_neg_centroid_cosine: 0.66, TL_A: 0.80
pos_pos_centroid_cosine: 0.86, pos_neg_centroid_cosine: 0.61, TL_P: 0.76
neg_pos_centroid_cosine: 0.86, neg_neg_centroid_cosine: 0.65, TL_N: 0.79
Epoch: 625
Loss: 0.82, pos_cosine: 0.71, neg_cosine: 0.64, TL: 0.93
anchor_pos_centroid_cosine: 0.84, anchor_neg_centroid

Epoch: 650
Loss: 0.80, pos_cosine: 0.72, neg_cosine: 0.64, TL: 0.92
anchor_pos_centroid_cosine: 0.86, anchor_neg_centroid_cosine: 0.64, TL_A: 0.78
pos_pos_centroid_cosine: 0.86, pos_neg_centroid_cosine: 0.60, TL_P: 0.74
neg_pos_centroid_cosine: 0.86, neg_neg_centroid_cosine: 0.64, TL_N: 0.78
Epoch: 651
Loss: 0.82, pos_cosine: 0.74, neg_cosine: 0.66, TL: 0.93
anchor_pos_centroid_cosine: 0.87, anchor_neg_centroid_cosine: 0.67, TL_A: 0.80
pos_pos_centroid_cosine: 0.86, pos_neg_centroid_cosine: 0.62, TL_P: 0.76
neg_pos_centroid_cosine: 0.87, neg_neg_centroid_cosine: 0.67, TL_N: 0.79
Epoch: 652
Loss: 0.83, pos_cosine: 0.72, neg_cosine: 0.66, TL: 0.94
anchor_pos_centroid_cosine: 0.87, anchor_neg_centroid_cosine: 0.66, TL_A: 0.79
pos_pos_centroid_cosine: 0.86, pos_neg_centroid_cosine: 0.62, TL_P: 0.77
neg_pos_centroid_cosine: 0.86, neg_neg_centroid_cosine: 0.67, TL_N: 0.80
Epoch: 653
Loss: 0.80, pos_cosine: 0.70, neg_cosine: 0.64, TL: 0.93
anchor_pos_centroid_cosine: 0.86, anchor_neg_centroid

In [ ]:
recall

In [ ]:
_[:20]

In [ ]:
# recall, exported_rank, debug = experiment.evaluate_validation_test(experiment, retrieval, verbose, 
#                                                         encoded_anchor, issues_by_buckets, evaluate_validation_test)
# test_vectorized, queries_test_vectorized, annoy, X_test, distance_test, indices_test = debug
# "recall@25 last epoch:", recall

### Retrieval evaluation

In [ ]:
print("Total of queries:", len(retrieval.test))

#### Getting the model trained

In [ ]:
SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs))

In [ ]:
model = encoded_anchor
# model = experiment.get_model_vectorizer(path=SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)))

In [ ]:
model.summary()

In [ ]:
recall, exported_rank, debug = experiment.evaluate_validation_test(retrieval, 0, model, issues_by_buckets, 
                                                                   bug_train_ids, method='bert')

In [ ]:
EXPORT_RANK_PATH = os.path.join(DIR, 'exported_rank_{}.txt'.format(METHOD))
EXPORT_RANK_PATH

In [ ]:
with open(EXPORT_RANK_PATH, 'w') as file_out:
    for row in exported_rank:
        file_out.write(row + "\n")

In [ ]:
report = experiment.evaluation.evaluate(EXPORT_RANK_PATH)
report